# Pair trading

The framework is straightforward:

- Step 1: Find cointergrated pairs
- Step 2: Backtesting each pair and find the best strategy
- Step 3: Evaluate and optimize the strategy

In [1]:
# Add the parent directory to the path so that we can import the module
import sys
sys.path.append("../")

In [2]:
# Main imports
from datetime import datetime
import pandas as pd
import numpy as np
from openbb import obb
from src.config import get_api_key
from src.utils import DataWrangler, TimeSeriesAnalysis
from src.analyzer import AnalyzerSuite
from src.data_manager import DataManager

In [3]:
# Initialize the API
pat = get_api_key()
obb.account.login(pat=pat)
dw = DataWrangler()
start_date = '2020-01-01'
end_date = '2023-01-01'

In [12]:
# Load equities list
tickers = DataManager().get_available_tickers()
# Test the loading data function
test_df = DataManager().load_ticker_data(tickers[0])
test_df.tail()

,open,high,low,close
date,,,,
2023-12-22,49.84,50.39,49.84,50.09
2023-12-26,50.11,50.40,50.05,50.28
2023-12-27,50.30,50.56,50.28,50.44
2023-12-28,50.58,50.63,50.42,50.48
2023-12-29,50.45,50.59,50.22,50.52


In [5]:
# Test cointegration function
apple_df = DataManager().load_ticker_data('AAPL')
msft_df = DataManager().load_ticker_data('MSFT')
tsa = TimeSeriesAnalysis()
tsa.adf_test(apple_df['adj_close'], msft_df['adj_close'])

In [ ]:
# Identify cointegrated pairs
pairs = []
# Loop through tickers list
for i in range(len(tickers)-1):
    for j in range(i+1, len(tickers)):
        # Load ticker data
        ticker1_df = DataManager().load_ticker_data(tickers[i])
        ticker2_df = DataManager().load_ticker_data(tickers[j])
        
        # Perform ADF test
        test = tsa.adf_test(ticker1_df['adj_close'], ticker2_df['adj_close'])
        # If cointegrated, add to pairs list
        if test:
            pairs.append((tickers[i], tickers[j]))

In [11]:
pairs

[('CSCO', 'BA'),
 ('CSCO', 'WBA'),
 ('CSCO', 'MSFT'),
 ('CSCO', 'HD'),
 ('BA', 'V'),
 ('BA', 'WBA'),
 ('BA', 'PG'),
 ('BA', 'CAT'),
 ('BA', 'MCD'),
 ('BA', 'MMM'),
 ('BA', 'MSFT'),
 ('BA', 'HD'),
 ('BA', 'AXP'),
 ('BA', 'NKE'),
 ('BA', 'IBM'),
 ('BA', 'GS'),
 ('BA', 'DIS'),
 ('BA', 'JPM'),
 ('BA', 'VZ'),
 ('BA', 'HON'),
 ('BA', 'AAPL'),
 ('BA', 'WMT'),
 ('BA', 'DOW'),
 ('BA', 'JNJ'),
 ('BA', 'CRM'),
 ('V', 'WBA'),
 ('V', 'WMT'),
 ('WBA', 'MSFT'),
 ('WBA', 'HD'),
 ('MCD', 'IBM'),
 ('MMM', 'CRM'),
 ('KO', 'IBM'),
 ('MSFT', 'HD'),
 ('MSFT', 'AXP'),
 ('MSFT', 'JPM'),
 ('AXP', 'AAPL'),
 ('IBM', 'AMGN'),
 ('IBM', 'JNJ')]